<a href="https://colab.research.google.com/github/liviabelirocha/webscraping-studies/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install python-slugify
!apt-get update
!apt install chromium-chromedriver

In [2]:
from selenium import webdriver
from slugify import slugify
import pandas as pd

In [48]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', options=options)

In [4]:
wd.get("https://www.thespruce.com/plants-a-to-z-5116344")
cards = wd.find_elements_by_class_name('alphabetical-card')

In [5]:
links = []
for card in cards:
  links.append(card.get_attribute('href'))
del links[:8]

In [45]:
def split_list(array, elem, type = 1):
  for idx, el in enumerate(array):
    if el == elem:
      return array[:idx] if type == 1 else array[idx+1:]

def convert_dict(array):  
    init = iter(array)
    res_dct = dict(zip(init, init))  
    return res_dct

def remove_words(array):
  for word in array:
    if word.replace(' ', '').isupper():
      array.remove(word)
  return array

def separate_arrays(array):
  titles = []
  contents = []
  aux_contents = []

  count = 0

  try:
    for elem in array:
      if len(elem.split(' ')) < 6:
        titles.append(elem)
        if count != 0:
          contents.append(aux_contents)
          aux_contents = []
        count += 1
      else:
        aux_contents.append(elem)
  except:
    return False, False

  return titles, contents

def handle_content(content):
  titles, contents = separate_arrays(content)

  new_titles = []
  new_contents = []

  if not titles and not contents:
    return False

  for (idx, content) in enumerate(contents):
    contents[idx] = ' '.join(content)

  for (idx, title) in enumerate(titles):
    slugs = ['light', 'soil', 'water', 'temperature-and-humidity', 'fertilizer']
    slug = slugify(title)

    if slug in slugs:
      new_titles.append(slug)
      new_contents.append(contents[idx])

  zip_iterator = zip(new_titles, new_contents)
  content_dict = dict(zip_iterator)
  return content_dict

def check_title(text):
  if text == 'native-areas':
    text = 'native-area'

  elif text == 'hardiness-zone' or text == 'usda-plant-hardiness-zones':
    text = 'hardiness-zones'

  elif text == 'latin-name':
    text = 'botanical-name'

  elif text == 'common-names':
    text = 'common-name'

  elif text == 'native-range':
    text = 'native-area'

  return text

In [46]:
def inspect_plant(link):
  wd.get(link)

  try:
    link = wd.find_element_by_xpath('/html/body/main/article/div[1]/div/div[2]/div[2]')
  except:
    return False

  page_list = link.text.split('\n')
  remove_words(page_list)

  try:
    table = link.find_element_by_tag_name('table')
  except:
    return False

  # Get description
  description = split_list(page_list, table.text.split('\n')[0])
  description = ''.join(description)

  content = split_list(page_list, table.text.split('\n')[-1], 2)
  content_dict = handle_content(content)

  if not content_dict:
    return False
  
  # Get table
  table = table.find_elements_by_tag_name('td')
  dict_table = []
  for (idx, column) in enumerate(table):
    text = column.text
    if idx % 2 == 0:
      text = slugify(text)
      text = check_title(text)
      
    dict_table.append(text)
  dict_table = convert_dict(dict_table)

  return {'description': description, **dict_table, **content_dict}

In [49]:
res = []
for link in links[:50]:
  plant = inspect_plant(link)
  if plant:
    res.append(plant)

df = pd.DataFrame(res)

In [52]:
del df['bloom-color']

In [57]:
df = df.dropna()

In [58]:
df

,description,botanical-name,common-name,plant-type,mature-size,sun-exposure,soil-type,soil-ph,bloom-time,flower-color,hardiness-zones,native-area,light,soil,water,temperature-and-humidity,fertilizer,toxicity
1,Aeonium is a genus including about 35 succulen...,Aeonium spp.,Aeonium,Perennial succulent,2–60 inches (depending on species and variety),Full sun to part shade,Sandy loam,5.6–6.0 (slightly acidic),Late winter or spring,"Pink (flowering is rare, occurring only in mat...",9–11(USDA); often grown as potted plants broug...,"Canary Islands, Africa","As with most succulents, Aeonium plants grow b...",A sandy loam or regular potting mix amended wi...,"In the winter, water whenever the top inch of ...",These plants prefer a Mediterranean climate—no...,Feed during the growing season with a half-str...,Non-toxic
2,The Aerangis genus comprises many tropical orc...,Aerangis spp.,Aerangis orchids,Epiphytic orchids,6–24 inches (varies by species),"Part sun, bright filtered light",Orchid potting mix,5.5 to 6.0 (acidic),Spring to fall (varies by species),"White, yellow",10–11 (USDA),"Tropical Africa, Madagascar",These tropical orchids need lots of bright ind...,"A well-draining epiphyte mix is best, such as ...",Aerangis orchids need a very moist environment...,These plants require warm tropical temperature...,"Feed regularly with a balanced, diluted fertil...",Non-toxic
3,African daisies (Osteospermum spp.) look a lot...,Osteospermum spp.,"African daisy, blue-eyed daisy, Cape daisy",Perennial,"1–3 ft. tall, 1–2 ft. wide",Full,"Moist, well-drained",Acidic,"Spring, summer, fall","Purple, pink, yellow, orange, white, bicolor",9–11 (USDA),"Africa, Asia",African daisies bloom best in full sun. They c...,African daisies prefer organically rich soil w...,Although somewhat drought tolerant once establ...,"African daisies prefer mild weather, which is ...",These flowers like a lot of food to grow and b...,Nontoxic
4,"African fern pine is a conifer with long, narr...",Afrocarpus gracilior or Podocarpus gracilior,African fern pine,Coniferous tree,"40–60 feet, 20–35 feet wide",Full sun (tolerates part shade),Well-draining,6.0 to 7.0 (slightly acidic to neutral),Spring (flowers are insignificant),Yellow,9–11 (USDA),East Africa,Fern pines do best in direct morning sunlight ...,African fern pines are not fussy when it comes...,"During the first two years, give a fern pine a...",The fern pine originated in the mountains of E...,Fertilize African fern pines in the springtime...,Possibly toxic
5,"Hailing from Southeast Asia, Amazon elephant's...",Alocasia x amazonica,"Amazon elephant's ear, African mask",Tropical,"1–2 ft. tall, 1–2 ft. wide",Partial shade,Moist but well-drained,Neutral to acidic,"Spring, summer (rarely blooms)",Green,9–11 (USDA),Asia,Amazon elephant's ear plants require lots of b...,"This plant prefers a fast-draining, well-aerat...",Keep the soil moist but remember that Amazon e...,"As a tropical plant, Amazon elephant's ear wil...",Amazon elephant's ear tends to be a heavy feed...,"Toxic to humans, dogs, and cats"
6,The African milk tree (Euphorbia trigona) is n...,Euphorbia trigona,"African Milk Tree, African Milk Bush",Succulent,6 to 8 ft.,Indirect sun to partial shade,"Medium rich, well-drained",6.1 to 7.8 (mildly acidic to mildly alkaline),"Spring, Summer",White (outdoors only),9b to 11 (USDA),Central Africa,This succulent likes indirect but bright sunli...,This plant is not too fussy about soil but goo...,"Being a succulent, the African milk tree doesn...",This drought-tolerant plant enjoys a dry or ar...,During the plant's growing season in the sprin...,Sap toxic
7,The African spear plant (Sansevieria cylindric...,Sansevieria cylindrica,"African spear plant, cylindrical snake plant, ...",Succulent,"4–6 ft. tall, 1–2 ft. wide","Full, partial","Sandy, well-drained",Neutral,Sporadic,White,10–11 (USDA),Africa,These plants can tolerate somewhat low light c...,"Like most succulents, these plants prefer a sa...",African spear plants c

In [59]:
from google.colab import files
df.to_csv('plants.csv', index=False) 
files.download('plants.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>